#### 1. Importamos las librerías necesarias.

In [1]:
import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyjsonviewer

import os
from dotenv import load_dotenv
load_dotenv()

False

#### 2. Creamos nuestras credenciales para acceder a nuestra lista de reproducción de Spotify.

In [2]:
# hemos dejado nuestros datos en .env, cifrado con .gitignore
client_ID = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

In [3]:
credenciales = SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret)

SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

In [ ]:
credenciales

In [ ]:
sp = spotipy.Spotify(client_credentials_manager=credenciales)

#### 3. Utilizamos el link de la lista para conseguir su identificador.

In [ ]:
link = "https://open.spotify.com/playlist/4PQ9xWUanbBuKwFFTc3482?si=265950455dd74ea8"

# utilizamos el metodo string para quedarnos sólo con el identificador de la lista de reproducción
iden = link.split("/")[-1].split("?")[0]
iden

'4PQ9xWUanbBuKwFFTc3482'

#### 4. Llamamos a la API para conseguir todos los datos de nuestras canciones.

In [ ]:
# Spotify solo nos devuelve 100 canciones con cada llamada, por lo que creamos una función que nos guarde en una variable toda nuestra data.
offset = 0
all_data = []
for i in range(8):
    all_data.append(sp.playlist_tracks(iden, offset=offset))
    offset += 100

In [ ]:
# Visualizamos todos los diccionarios en una ventana externa para mejorar su legibilidad.
pyjsonviewer.view_data(json_data=all_data)

In [ ]:
all_data[1]["items"][60].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

#### 5. Comprobamos el funcionamiento de las listas de diccionarios para buscar información.

In [ ]:
# cuando se agregó la canción
all_data[1]["items"][9]['added_at']

'2022-11-22T14:26:51Z'

In [ ]:
# qué usuario la agregó
all_data[1]["items"][9]['added_by']['id']

'31p3oc6g3fcxi2hep3pg6e25aifm'

In [ ]:
all_data[1]["items"][9]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [ ]:
# nombre la canción
all_data[1]["items"][9]['track']['name']

'Every Breath You Take'

In [ ]:
# nombre del artista
all_data[1]["items"][9]['track']['artists'][0]['name']

'The Police'

In [ ]:
all_data[1]["items"][9]['track']['album'].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [ ]:
# album de la canción
all_data[1]["items"][9]['track']['album']['name']

'Synchronicity (Remastered 2003)'

In [ ]:
# cuando se estrenó la canción
all_data[1]["items"][9]['track']['album']['release_date']

'1983-06-17'

In [ ]:
# portada del album
all_data[1]["items"][9]['track']['album']['images'][0]['url']

'https://i.scdn.co/image/ab67616d0000b273c8e97cafeb2acb85b21a777e'

In [ ]:
# popularidad de la canción
all_data[1]["items"][9]['track']['popularity']

87

In [ ]:
# duración
all_data[1]["items"][9]['track']['duration_ms']

253920

In [ ]:
# explícita
all_data[1]["items"][9]['track']['explicit']

False

In [ ]:
# identificador de la canción
all_data[1]["items"][9]['track']['uri']

'spotify:track:1JSTJqkT5qHq8MDJnJbRE1'

#### 6. Creamos una función para iterar por toda la lista de reproducción.

In [ ]:
basic_info = {"song": [], 
            "artist": [], 
            "album": [], 
            "duration": [],
            "date_add": [],
            "date_album": [], 
            "explicit": [], 
            "popularity": [], 
            "user": [], 
            "image": [],  
            "uri": [] }

for diccionario in range(len(all_data)):
    for cancion in range(len(all_data[diccionario]["items"])):
        basic_info["song"].append(all_data[diccionario]["items"][cancion]["track"]["name"])
        basic_info["album"].append(all_data[diccionario]["items"][cancion]["track"]["album"]["name"])
        basic_info["duration"].append(all_data[diccionario]["items"][cancion]["track"]["duration_ms"])
        basic_info["date_add"].append(all_data[diccionario]["items"][cancion]["added_at"])
        basic_info["date_album"].append(all_data[diccionario]["items"][cancion]["track"]["album"]["release_date"])
        basic_info["explicit"].append(all_data[diccionario]["items"][cancion]["track"]["explicit"])
        basic_info["popularity"].append(all_data[diccionario]["items"][cancion]["track"]["popularity"])
        basic_info["user"].append(all_data[diccionario]["items"][cancion]["added_by"]["id"])
        basic_info["image"].append(all_data[diccionario]["items"][cancion]["track"]["album"]["images"][0]["url"])
        basic_info["uri"].append(all_data[diccionario]["items"][cancion]["track"]["uri"])


        numero_artistas = all_data[diccionario]["items"][cancion]["track"]["artists"]

        if len(numero_artistas) == 1:
            basic_info["artist"].append(all_data[diccionario]["items"][cancion]["track"]["artists"][0]["name"])
        else:
            lista_artista = []
            for artista in range(len(numero_artistas)):
                lista_artista.append(all_data[diccionario]["items"][cancion]["track"]["artists"][artista]["name"])
            basic_info["artist"].append(lista_artista)

In [ ]:
basic_info['song'][2]

"Hard to Say I'm Sorry"

In [ ]:
basic_info['artist'][2]

'Chicago'

In [ ]:
basic_info['user'][2]

'laurezurich'

#### 7. Pasamos la lista de diccionarios a DataFrame para trabajar con él.

In [ ]:
# comprobamos que todas las columnas tienen la misma medida
for k,v in basic_info.items():
    print(k, "--", len(v))

song -- 677
artist -- 677
album -- 677
duration -- 677
date_add -- 677
date_album -- 677
explicit -- 677
popularity -- 677
user -- 677
image -- 677
uri -- 677


In [ ]:
df = pd.DataFrame(basic_info)

: 

#### 7. Como la mayoría de los ID de los users son irreconocibles, modificamos los nombres.